Using Multiple Tools in Langchain

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the product."""
    return a * b

def translate_to_french(text: str) -> str:
    """Return a mock French translation of the input text."""
    return f"French translation of '{text}'"

def get_weather(city: str) -> str:
    """Return a mock weather string for the given city."""
    return f"The weather in {city} is sunny."

LLM + Tools Binding

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5")

In [ ]:
llm_with_tools = llm.bind_tools([multiply, translate_to_french, get_weather])

LangGraph Workflow

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
# State that holds conversation history
class MyMessagesState(MessagesState):
    pass

In [ ]:
# Node: the AI generates a response and may call a tool
def tool_calling_llm(state: MyMessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
# Build the graph
builder = StateGraph(MyMessagesState)

In [ ]:

# Add nodes
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply, translate_to_french, get_weather]))

In [ ]:
# Connect nodes
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", END)

In [ ]:
# Compile
graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
messages = graph.invoke({"messages": [HumanMessage(content="Multiply 4 and 5")]})
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": [HumanMessage(content="What's the weather in Paris?")]})
for m in messages["messages"]:
    m.pretty_print()